# Homework 5 - Explore California and Nevada with graphs
---
Here we import every library we'll need.

In [1]:
import networkx as nx
import numpy as np
from collections import deque

---
## Graph class
Here we're going to create our graph class which we're going to use for basically every task. Since we'll use a lot of nodes and every node shouldn't be connected to many edges (considering that our graph represents a map of roads) we'll use an djacency list to represent the graph. An adjacency matrix would take too much memory and most of the entries would be zero, so it would be really inefficient.

We're going to assume we'll work with undirected graphs, so we implement a class consequentially.

In [2]:
# This class is going to represent a graph, so that we can organize all the implementation and methods of
# our data structure in an organized way
class Graph:
    
    # This constructur takes as parameter the number of nodes
    def __init__(self, size):
        self._nodes = [[] for _ in range(size)]
        self._nodes_features = [{} for _ in range(size)]
        
    # This method adds features to a node in the form of a dictionary
    def add_features_to_node(self, node, features, replace = False):
        if(replace):
            self._nodes_features[node - 1] = features
        else:
            for key in features.keys():
                self._nodes_features[node - 1][key] = features[key]
                
    # This method is for removing all given features from the nodes of the graph, or all features if no second
    # parameter is given
    def reset_features(self, features = None):
        if(features is None):
            for index in range(len(self._nodes_features)):
                self._nodes_features[index] = {}
        else:
            for index in range(len(self._nodes_features)):
                for feature in features:
                    self._nodes_features[index].pop(feature, None)
        
    # This method is for getting features of a node
    def get_feature_of_node(self, node, feature):
        moment = self._nodes_features[node - 1]
        if(feature in moment.keys()):
            return(moment[feature])
        return(None)
        
    # This method adds an edge to the graph, together with an optional dict of weights
    # The edge is given as a tuple of nodes
    def add_edge(self, edge, weights = None):
        to_add = (edge[1] - 1, {})
        if(not weights is None):
            to_add = (edge[1] - 1, weights)
        self._nodes[edge[0] - 1].append(to_add)
        
        to_add = (edge[0] - 1, {})
        if(not weights is None):
            to_add = (edge[0] - 1, weights)
        self._nodes[edge[1] - 1].append(to_add)
        
    # This method gets all neighbours of a node and given weights of the edges connecting them to the node
    def get_neighbours(self, node, features = None):
        if(features is None):
            return([single_node[0] + 1 for single_node in self._nodes[node - 1]])
        to_return = []
        for neighbour in self._nodes[node - 1]:
            to_return.append((neighbour[0] + 1, dict((k, neighbour[1][k]) for k in features if k in neighbour[1])))
        return(to_return)
    
    # This method get an edge from the graph together with the specified weights
    # If the edge doesn't exist it returns None
    def get_edge(self, node_one, node_two, features = None):
        for neighbour in self._nodes[node_one - 1]:
            if(neighbour[0] == node_two - 1):
                if(features is None):
                    return((node_one, node_two))
                return((node_one, node_two, dict((k, neighbour[1][k]) for k in features if k in neighbour[1])))
        return(None)
    
    # This method updates and edge adding more weights or replacing them (based on replace parameter)
    # It doesn't do anything if the edge doesn't exist
    def update_edge(self, node_one, node_two, weights):
        for neighbour in self._nodes[node_one - 1]:
            if(neighbour[0] == node_two - 1):
                for key in weights.keys():
                    neighbour[1][key] = weights[key]
                return
            
    # This method prints the adjacency list of the graph, it will be used just to check that everything works fine
    def print_graph(self, head = None):
        if(head is None):
            head = len(self._nodes)
        for index in range(head):
            print(index + 1, end = " : ")
            print([(node[0] + 1, node[1]) for node in self._nodes[index]])

---
## Collecting data
Here we're going to use the files we have downloaded to create and populate our graph of interest. To start we need to count how many nodes we need so we can correctly create our graph.

In [3]:
counter = 0
nodes = open("USA-road-d.CAL.co" , "r")
while(True):
    string = nodes.readline()
    if(string == ""):
        break
    if(string[0] == 'v'):
        counter = counter + 1
nodes.close()
print(counter)

1890815


Now that we know our graph will contain $1890815$ nodes we can create it.

In [4]:
CAL = Graph(1890815)

Now we're going to populate our graph wiht edges. Since every edge appear four times in total in our files (twice per file), we need some caution in doing this. In fact we have to be aware that our graph doesn't see if an edge already exists before adding it and since it's an undirected graph class we don't need to add every edge twice (one for direction).

In [5]:
times = open("USA-road-t.CAL.gr", "r")
distances = open("USA-road-d.CAL.gr", "r")
counter = 0
while(True):
    string_time = times.readline()
    string_distance = distances.readline()
    if(string_time == ""):
        break
    if(string_time[0] == 'a' and (counter % 2) == 1):
        edge_time = string_time.split(" ")
        edge_distance = string_distance.split(" ")
        CAL.add_edge((int(edge_time[1]), int(edge_time[2])), {"Time" : int(edge_time[3]), "Distance" : int(edge_distance[3])})  
    counter = counter + 1
times.close()
distances.close()

---
## Shortest path in a graph
In order to solve a lot of tasks we'll need a way to find the shortest path connecting two nodes in a graph. This is a common problem and we'll implement two different algorithms to accomplish our goal.

The first one is the breadth-first-search to find out what the minimum path connecting two nodes in a graph is, whitout taking into consideration any weight. Then we'll implement the Dijkstra's algorithm to find the shortest path based on a certain weight.

### Breadth-first-search
Here we implement our breadth-first-search algorithm. Since we want it to work for graphs belonging to our class we're going to implement this method like it belongs to our <code>Graph</code> class and then use <code>seatattr()</code> to actually add it to the class.

In [6]:
# This method finds the shortest path connecting a source and a target node using a breadth-first-search
# It returns None if not such a path exists
def breadth_first_search(self, source, target):
    in_queue_nodes = deque()
    
    in_queue_nodes.appendleft(source)
    self.add_features_to_node(source, {"Visited" : True})
    while(True):
        if(not in_queue_nodes):
            self.reset_features(["Visited", "Parent"])
            return(None)
        visiting_now = in_queue_nodes.pop()
        if(visiting_now == target):
            break
        for neighbour in self.get_neighbours(visiting_now):
            if(self.get_feature_of_node(neighbour, "Visited") is None):
                in_queue_nodes.appendleft(neighbour)
                self.add_features_to_node(neighbour, {"Parent" : visiting_now, "Visited" : True})
    
    to_return = []
    while(not visiting_now is None):
        to_return.append(visiting_now)
        visiting_now = self.get_feature_of_node(visiting_now, "Parent")
    self.reset_features(["Visited", "Parent"])
    
    return(to_return[::-1])

setattr(Graph, 'breadth_first_search', breadth_first_search)

### Dijkstra algorithm
Here we implement our Dijkstra algorithm which we'll use to find the shortest path in a graph connecting two nodes taking into considerations weights. Before we can code the algorithm we need an heap to store nodes and extracting them.

We're going to use a binaryl heap, which should be enough to make an efficient algorithm. We could use a Binomial heap or a Fibonacci heap, but it would be a lot more complicated and, considered constant factors, it's not even guaranteed to gain much efficiency.

So let's start defining our <code>Binary_Heap</code> class. We're going to represent it as a list <code>heap</code> where every entry is a node and children of <code>heap[i]</code> are <code>heap[2i]</code> and <code>heap[2i + 1]</code>. The binary heap is associated with an <code>int_function</code> which converts nodes to integer used for comparison.

In [7]:
# This class represents a binary heap using a list.
class Binary_Heap:
    
    # This constructor is for creating a binary heap containing a single node
    def __init__(self, single_node, int_function = lambda x : x):
        self._heap = [None, single_node]
        self._int_function = int_function
    
    # This constructor is for getting left child of a node given his index, None if not such a child exists
    def get_left_child(self, node_index):
        position = node_index * 2
        if(position >= len(self._heap)):
            return(None)
        return(self._heap[position])
    
    # Like left child, but for right child
    def get_right_child(self, node_index):
        position = (node_index * 2) + 1
        if(position >= len(self._heap)):
            return(None)
        return(self._heap[position])
    
    # This constructor is for getting parent of a node given is index, None if not such a parent exists
    def get_parent(self, node_index):
        return(self._heap[node_index // 2])
    
    # This is the heapify function, it helps preserving the properties of the heap
    def heapify(self, index):
        left = 2 * index
        right = (2 * index) + 1
        minimum = index
        if(left < len(self._heap) and self._int_function(self._heap[left]) < self._int_function(self._heap[minimum])):
            minimum = left
        if(right < len(self._heap) and self._int_function(self._heap[right]) < self._int_function(self._heap[minimum])):
            minimum = right
        if(minimum != index):
            aux = self._heap[index]
            self._heap[index] = self._heap[minimum]
            self._heap[minimum] = aux
            self.heapify(minimum)
        
    # This function is for inserting a new node inside the heap
    def insert(self, new_node):
        if(len(self._heap) == 1):
            self._heap.append(new_node)
            return
        self._heap.append(new_node)
        index = len(self._heap) - 1
        while(self._int_function(self._heap[index]) < self._int_function(self.get_parent(index))):
            aux = self._heap[index]
            self._heap[index] = self.get_parent(index)
            self._heap[index // 2] = aux
            index = index // 2
            if(index // 2 == 0):
                return
    
    # This function is for extracting the minimum element from the heap
    def extract(self):
        if(len(self._heap) == 2):
            return(self._heap.pop())
        to_return = self._heap[1]
        self._heap[1] = self._heap.pop()
        self.heapify(1)
        return(to_return)
    
    # Method for checking if the heap is empty
    def is_empty(self):
        return(len(self._heap) == 1)

Now we can implement our Dijkstra algorithm.

In [8]:
# Dijkstra's algorithm, it finds the shortest path in a graph connecting source and target taking into consideration
# the given weight.
def dijkstra(self, source, target, weight):
    heap = Binary_Heap((source, 0), lambda x : x[1])
    self.add_features_to_node(source, {"Distance" : 0})
    while(True):
        if(heap.is_empty()):
            self.reset_features(["Visited", "Distance", "Parent"])
            return(None)
        current_node = heap.extract()
        if(current_node[0] == target):
            break
        if(self.get_feature_of_node(current_node[0], "Visited")):
            continue
        this_distance = self.get_feature_of_node(current_node[0], "Distance")
        for close_node in self.get_neighbours(current_node[0], [weight]):
            prev_distance = self.get_feature_of_node(close_node[0], "Distance")
            arc_weight = close_node[1][weight]
            new_distance = this_distance + arc_weight
            if(prev_distance is None or prev_distance > new_distance):
                self.add_features_to_node(close_node[0], {"Distance" : new_distance, "Parent" : current_node[0]})
                heap.insert((close_node[0], new_distance))
        self.add_features_to_node(current_node[0], {"Visited" : True})
    
    current_node = current_node[0]
    to_return = []
    while(not current_node is None):
        to_return.append(current_node)
        current_node = self.get_feature_of_node(current_node, "Parent")
    self.reset_features(["Visited", "Distance", "Parent"])
    return(to_return[::-1])

setattr(Graph, 'dijkstra', dijkstra)

---
## Functionality 3 - Shortest Ordered Route
In this section we're going to implement functionality 3. Most of the work associated to it comes from finding the shortest path in a graph connecting two nodes and since we have already done that this will be a quick task.

We start from a simple, but fundamental observation: given a node $H$ and a sequence of nodes $p_{1}, \dots, p_{n}$ let $s_{i}$ be the shortest path going from $H$ to $p_{n}$ as asked by the functionality ($s_{i}$ is the $i$-th node taken by the path). Then for every couple of consecutive nodes $p_{j}, p_{j + 1}$ (or $H, p_{1}$) the subpath of $s_{i}$ connecting $p_{j}$ and $p_{j + 1}$ is the shortest path in the graph connecting $p_{j}$ and $p_{j + 1}$. In fact, if that's not the case we could substitute this subpath with the shortest path and we would get a shorter path than $s_{i}$ satisfying our properties.

Considered this, to find the path asked by the functionality we just need to find the shortest path connecting two nodes sequentially: first for $H$ and $p_{1}$, then for $p_{1}$ and $p_{2}$ etc. etc.

So let's do this. As always we're going to code a function and then we'll add it to our <code>Graph</code> class with the <code>setattr</code> method.

In [9]:
# This class finds the shortest ordered route given a sequence of nodes and based on a certain distance, if no
# distance is specified it uses the network distance
# The method uses Dijkstra when a distance is specified or Breadth-First-Search for network distance
# The function returns None if not such a route exists
def shortest_ordered_route(self, H, sequence_of_nodes, distance = None):
    route = [H]
    sequence = [H]
    sequence.extend(sequence_of_nodes)
    for index in range(len(sequence) - 1):
        if(distance is None):
            sub_route = self.breadth_first_search(sequence[index], sequence[index + 1])
        else:
            sub_route = self.dijkstra(sequence[index], sequence[index + 1], distance)
        if(sub_route is None):
            return(None)
        route.extend(sub_route[1:])
    return(route)

setattr(Graph, 'shortest_ordered_route', shortest_ordered_route)